## LLMs and augmentations

Workflows and agentic systems are based on LLMs and the various augmentations you add to them. [Tool calling](https://docs.langchain.com/oss/python/langchain/tools), [structured outputs](https://docs.langchain.com/oss/python/langchain/structured-output), and [short term memory](https://docs.langchain.com/oss/python/langchain/short-term-memory) are a few options for tailoring LLMs to your needs.

<img src="https://mintcdn.com/langchain-5e9cc07a/-_xGPoyjhyiDWTPJ/oss/images/augmented_llm.png?fit=max&auto=format&n=-_xGPoyjhyiDWTPJ&q=85&s=7ea9656f46649b3ebac19e8309ae9006" alt="LLM augmentations" data-path="oss/images/augmented_llm.png" data-optimize="true" data-opv="3" srcset="https://mintcdn.com/langchain-5e9cc07a/-_xGPoyjhyiDWTPJ/oss/images/augmented_llm.png?w=280&fit=max&auto=format&n=-_xGPoyjhyiDWTPJ&q=85&s=53613048c1b8bd3241bd27900a872ead 280w, https://mintcdn.com/langchain-5e9cc07a/-_xGPoyjhyiDWTPJ/oss/images/augmented_llm.png?w=560&fit=max&auto=format&n=-_xGPoyjhyiDWTPJ&q=85&s=7ba1f4427fd847bd410541ae38d66d40 560w, https://mintcdn.com/langchain-5e9cc07a/-_xGPoyjhyiDWTPJ/oss/images/augmented_llm.png?w=840&fit=max&auto=format&n=-_xGPoyjhyiDWTPJ&q=85&s=503822cf29a28500deb56f463b4244e4 840w, https://mintcdn.com/langchain-5e9cc07a/-_xGPoyjhyiDWTPJ/oss/images/augmented_llm.png?w=1100&fit=max&auto=format&n=-_xGPoyjhyiDWTPJ&q=85&s=279e0440278d3a26b73c72695636272e 1100w, https://mintcdn.com/langchain-5e9cc07a/-_xGPoyjhyiDWTPJ/oss/images/augmented_llm.png?w=1650&fit=max&auto=format&n=-_xGPoyjhyiDWTPJ&q=85&s=d936838b98bc9dce25168e2b2cfd23d0 1650w, https://mintcdn.com/langchain-5e9cc07a/-_xGPoyjhyiDWTPJ/oss/images/augmented_llm.png?w=2500&fit=max&auto=format&n=-_xGPoyjhyiDWTPJ&q=85&s=fa2115f972bc1152b5e03ae590600fa3 2500w" />


## Structured Output

Structured outputs are a way to extract structured data from the LLM's response.

In [ ]:
# Schema for structured output
from pydantic import BaseModel, Field


class SearchQuery(BaseModel):
    search_query: str = Field(None, description="Query that is optimized web search.")
    justification: str = Field(
        None, description="Why this query is relevant to the user's request."
    )

# Augment the LLM with schema for structured output
structured_llm = llm.with_structured_output(SearchQuery)

# Invoke the augmented LLM
output = structured_llm.invoke("How does Calcium CT score relate to high cholesterol?")

/home/halgoz/work/ai-agents/content/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='parsed', input_value=SearchQuery(search_query=...-to-date explanations.'), input_type=SearchQuery])
  return self.__pydantic_serializer__.to_python(


In [ ]:
print("search_query:", output.search_query)
print("justification:", output.justification)

search_query: Calcium CT score relationship with high cholesterol atherosclerosis risk
justification: The user wants to understand how a coronary artery calcium (CAC) score derived from CT imaging connects to elevated blood cholesterol levels. This requires information on the pathophysiology of atherosclerotic plaque formation, the role of lipid disorders, and how CAC scoring is used clinically to assess cardiovascular risk in the context of cholesterol. The query targets medical literature and guidelines that discuss these relationships, making it the most relevant search term for retrieving accurate, up-to-date explanations.


## Tool Calling

Tools connect the LLM to the outside world.

In [ ]:
from langchain.tools import tool

# Define a tool
@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers."""
    return a * b

In [ ]:
# Bind (potentially multiple) tools to the model
llm_with_tools = llm.bind_tools([multiply])

Step 1: Model generates tool calls

In [ ]:
messages = [{"role": "user", "content": "What is 2 times 3"}]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

In [ ]:
messages

[{'role': 'user', 'content': 'What is 2 times 3'},
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 295, 'total_tokens': 414, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 82, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'cost': 0, 'is_byok': False, 'cost_details': {'upstream_inference_cost': 0, 'upstream_inference_prompt_cost': 0, 'upstream_inference_completions_cost': 0}}, 'model_provider': 'openai', 'model_name': 'nvidia/nemotron-3-nano-30b-a3b:free', 'system_fingerprint': None, 'id': 'gen-1771681856-uclSTv2ef3GhPvPUeA98', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c8077-b908-7fd1-aa62-f083b42d90f6-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_fb2407be30fa4f989ea9a4c6', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata=

In [ ]:
# Get the tool call
messages[-1].tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': 'call_fb2407be30fa4f989ea9a4c6',
  'type': 'tool_call'}]

Step 2: Execute tools and collect results

In [ ]:
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = multiply.invoke(tool_call)
    messages.append(tool_result)

In [ ]:
messages

[{'role': 'user', 'content': 'What is 2 times 3'},
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 295, 'total_tokens': 414, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 82, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'cost': 0, 'is_byok': False, 'cost_details': {'upstream_inference_cost': 0, 'upstream_inference_prompt_cost': 0, 'upstream_inference_completions_cost': 0}}, 'model_provider': 'openai', 'model_name': 'nvidia/nemotron-3-nano-30b-a3b:free', 'system_fingerprint': None, 'id': 'gen-1771681856-uclSTv2ef3GhPvPUeA98', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c8077-b908-7fd1-aa62-f083b42d90f6-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_fb2407be30fa4f989ea9a4c6', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata=

Step 3: Pass results back to model for final response

In [ ]:
final_response = llm_with_tools.invoke(messages)
print(final_response.text)

The product of 2 and 3 is 6. 

<final_answer>
6
</final_answer>


Essentially, this is what the implementation of the agent resulting from `create_agent`, in langchain is all about.